In [10]:
pip install accelerate==0.20.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.5/227.5 kB 3.2 MB/s eta 0:00:00


In [12]:
pip install transformers[torch]==4.15.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 17.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: Could not build wheels for tokenizers, which is required to install pyproject.toml-based projects


In [16]:
pip install cython
pip install -U setuptools
pip install transformers[torch]==4.15.0


SyntaxError: ignored

In [17]:
import pandas as pd
df=messages = pd.read_csv('/content/LLM-Sample-Input-File.csv')
df.head()

,Company Name,Category,Sub Cat,Period,Value - Randomized
0,Potato Inc.,Revenue By Region,Americas,2015 Q1,1.830000e+11
1,Potato Inc.,Revenue By Region,Americas,2015 Q2,1.710000e+11
2,Potato Inc.,Revenue By Region,Americas,2015 Q3,8.084000e+10
3,Potato Inc.,Revenue By Region,Americas,2015 Q4,6.531000e+10
4,Potato Inc.,Revenue By Region,Americas,2016 Q1,2.350000e+11


In [ ]:
X=list(df['message'])

In [ ]:
y=list(df['label'])

In [21]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config
from torch.utils.data import Dataset, DataLoader
from torch import nn
import torch

# Load your CSV file into a DataFrame
# Replace 'your_dataset.csv' with the actual file path
df = pd.read_csv('/content/LLM-Sample-Input-File.csv')

# Assuming your CSV has a column named 'Company Name' containing text data
texts = df['Company Name'].tolist()

# Tokenize the texts
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token  # Set padding token
tokenized_texts = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=512)

# Create a custom dataset
class CustomDataset(Dataset):
    def __init__(self, tokenized_texts):
        self.input_ids = tokenized_texts['input_ids']
        self.attention_mask = tokenized_texts['attention_mask']

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
        }

dataset = CustomDataset(tokenized_texts)

# Create a DataLoader
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Load the pre-trained GPT-2 model
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Define a custom head for fine-tuning
class CustomHead(nn.Module):
    def __init__(self):
        super(CustomHead, self).__init__()
        self.linear = nn.Linear(model.config.hidden_size, tokenizer.vocab_size)

    def forward(self, x):
        return self.linear(x)

# Replace the language model head with the custom head
model.lm_head = CustomHead()

# Set up training parameters
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
criterion = nn.CrossEntropyLoss()

# Fine-tune the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.train()

num_epochs = 3
for epoch in range(num_epochs):
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        labels = input_ids.clone()
        labels[labels == tokenizer.pad_token_id] = -100  # Ignore padding in the loss

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

# Save the fine-tuned model
model.save_pretrained('fine_tuned_model')
tokenizer.save_pretrained('fine_tuned_model')


Epoch 1/3, Loss: 3.897705078125
Epoch 2/3, Loss: 0.04975556209683418
Epoch 3/3, Loss: 0.014728009700775146


('fine_tuned_model/tokenizer_config.json',
 'fine_tuned_model/special_tokens_map.json',
 'fine_tuned_model/vocab.json',
 'fine_tuned_model/merges.txt',
 'fine_tuned_model/added_tokens.json')

In [22]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Load the fine-tuned GPT-2 model and tokenizer
model = GPT2LMHeadModel.from_pretrained('fine_tuned_model')
tokenizer = GPT2Tokenizer.from_pretrained('fine_tuned_model')

# Set the model to evaluation mode
model.eval()

# Define a function for the chatbot
def chatbot(input_text):
    # Tokenize the input text
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # Generate a response from the model
    with torch.no_grad():
        output = model.generate(input_ids, max_length=100, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95, temperature=0.7)

    # Decode the generated response
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    return response

# Example conversation with the chatbot
print("Chatbot: Hello! How can I help you today?")
while True:
    user_input = input("You: ")

    if user_input.lower() in ['exit', 'quit', 'bye']:
        print("Chatbot: Goodbye!")
        break

    response = chatbot(user_input)
    print("Chatbot:", response)


Some weights of the model checkpoint at fine_tuned_model were not used when initializing GPT2LMHeadModel: ['lm_head.linear.weight', 'lm_head.linear.bias']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Chatbot: Hello! How can I help you today?
You: How much revenue does Potato Inc. make from selling Smartphones


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Chatbot: How much revenue does Potato Inc. make from selling Smartphones and tablets?

PATO INC. is a registered investment company registered in the U.S. and other countries. It is not a publicly traded company and does not make any financial statements. The information in this press release is provided "as is" and solely for informational purposes, not for trading purposes or advice. Neither the TSX Venture Exchange nor its Regulation Services Provider ("TSX") accepts responsibility for the adequacy or
You: How much revenue does Potato Inc. make from selling Smartphones


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Chatbot: How much revenue does Potato Inc. make from selling Smartphones and tablets?

PATO INC. is a registered investment company registered in the U.S. and other countries. It is not a publicly traded company and does not make any financial statements. The information in this press release is provided "as is" and solely for informational purposes, not for trading purposes or advice. Neither the TSX Venture Exchange nor its Regulation Services Provider ("TSX") accepts responsibility for the adequacy or
You: How much revenue does Potato Inc. make from Japan


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Chatbot: How much revenue does Potato Inc. make from Japan? View On reddit.com submitted 1 year ago by murrayyyyy posted in /r/KotakuInAction

This poll has been closed.
You: bye
Chatbot: Goodbye!
